In [1]:
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

def cost_curve(x,premium,contribution,deductible,coinsuranceRate,maxOutOfPocket):
    eff_premium = premium - contribution
    post_deduct = (coinsuranceRate * (x - deductible)) + deductible
    y = np.fmin(np.fmin(x, post_deduct), maxOutOfPocket) + eff_premium
    return y

In [2]:
x = np.linspace(0.0, 10000, 100)

premium1 = Slider(start=0, end=2000, value=100, step=.1, title="Premium")
contribution1 = Slider(start=0, end=1000, value=50, step=10, title="Company Contribution")
deductible1 = Slider(start=0, end=6000, value=2000, step=10, title="Deductible")
coinsurance1 = Slider(start=0, end=1, value=0.5, step=.01, title="Coinsurance Rate (paid by you)")
maxOutOfPocket1 = Slider(start=0, end=8000, value=4000, step=10, title="Maximum Out Of Pocket")
y1 = cost_curve(x,premium1.value,contribution1.value,deductible1.value,coinsurance1.value,maxOutOfPocket1.value)

premium2 = Slider(start=0, end=2000, value=100, step=.1, title="Premium")
contribution2 = Slider(start=0, end=1000, value=50, step=10, title="Company Contribution")
deductible2 = Slider(start=0, end=6000, value=2000, step=10, title="Deductible")
coinsurance2 = Slider(start=0, end=1, value=0.5, step=.01, title="Coinsurance Rate (paid by you)")
maxOutOfPocket2 = Slider(start=0, end=8000, value=4000, step=10, title="Maximum Out Of Pocket")
y2 = cost_curve(x,premium2.value,contribution2.value,deductible2.value,coinsurance2.value,maxOutOfPocket2.value)

plot = figure(x_range=(0,10000), y_range=(0,10000), plot_width=400, plot_height=400, tools="pan,hover,crosshair,reset")
plot.line(x, x, line_dash=(4,4), line_color="black")
plot.xaxis.axis_label = 'Out-of-Pocket Expenses'
plot.yaxis.axis_label = 'Actual Incurred Cost'

source1 = ColumnDataSource(data=dict(x=x, y=y1))
source2 = ColumnDataSource(data=dict(x=x, y=y2))
plot.line('x', 'y', source=source1, line_width=3, line_alpha=0.6, legend="Plan 1")
plot.line('x', 'y', source=source2, line_width=3, line_alpha=0.6, legend="Plan 2", line_color="orange")

callback1 = CustomJS(args=dict(source=source1, premium=premium1, contribution=contribution1, deductible=deductible1, coinsuranceRate=coinsurance1, maxOutOfPocket=maxOutOfPocket1),
                    code="""
    const data = source.data;
    const P = premium.value - contribution.value;
    const D = deductible.value;
    const R = coinsuranceRate.value;
    const M = maxOutOfPocket.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        const post_deduct = (R*(x[i] - D)) + D;
        y[i] = Math.min(Math.min(x[i],post_deduct),M) + P;
    }
    source.change.emit();
""")

callback2 = CustomJS(args=dict(source=source2, premium=premium2, contribution=contribution2, deductible=deductible2, coinsuranceRate=coinsurance2, maxOutOfPocket=maxOutOfPocket2),
                    code="""
    const data = source.data;
    const P = premium.value - contribution.value;
    const D = deductible.value;
    const R = coinsuranceRate.value;
    const M = maxOutOfPocket.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        const post_deduct = (R*(x[i] - D)) + D;
        y[i] = Math.min(Math.min(x[i],post_deduct),M) + P;
    }
    source.change.emit();
""")

premium1.js_on_change('value', callback1)
contribution1.js_on_change('value', callback1)
deductible1.js_on_change('value', callback1)
coinsurance1.js_on_change('value', callback1)
maxOutOfPocket1.js_on_change('value', callback1)

premium2.js_on_change('value', callback2)
contribution2.js_on_change('value', callback2)
deductible2.js_on_change('value', callback2)
coinsurance2.js_on_change('value', callback2)
maxOutOfPocket2.js_on_change('value', callback2)

layout = row(
    plot,
    column(premium1, contribution1, deductible1, coinsurance1, maxOutOfPocket1),
    column(premium2, contribution2, deductible2, coinsurance2, maxOutOfPocket2),
)

output_file("main.html", title="Insurance Plan Comparison")

show(layout)